In [1]:
getwd()


[1] "C:/Users/Douglas Schoewe"

In [3]:
setwd("C:\\rscripts")

In [33]:
tract_311 <- read.csv("./data/311_Tract_Coded.csv")

In [34]:
head(tract_311)

,GEOID10,Category
,<dbl>,<chr>
1,6075980600,Abandoned Vehicle
2,6075980600,Graffiti Public Property
3,6075980600,Street and Sidewalk Cleaning
4,6075980600,Street and Sidewalk Cleaning
5,6075980600,Street and Sidewalk Cleaning
6,6075980600,Street and Sidewalk Cleaning


In [35]:
WAC <- read.csv("./data/ca_wac_SI03_JT00_2014.csv")

In [36]:
options(scipen = 999)

In [37]:
head(WAC)

,w_geocode,C000,CA01,CA02,CA03,CE01,CE02,CE03,CNS01,CNS02,⋯,CFA02,CFA03,CFA04,CFA05,CFS01,CFS02,CFS03,CFS04,CFS05,createdate
,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,60014001001007,45,17,15,13,16,7,22,0,0,⋯,0,0,0,0,0,0,0,0,0,20190825
2,60014001001008,26,3,15,8,3,5,18,0,0,⋯,0,0,0,0,0,0,0,0,0,20190825
3,60014001001017,9,2,3,4,1,1,7,0,0,⋯,0,0,0,0,0,0,0,0,0,20190825
4,60014001001024,19,9,9,1,9,6,4,0,0,⋯,0,0,0,0,0,0,0,0,0,20190825
5,60014001001026,4,1,2,1,0,3,1,0,0,⋯,0,0,0,0,0,0,0,0,0,20190825
6,60014001001027,11,2,1,8,5,4,2,0,0,⋯,0,0,0,0,0,0,0,0,0,20190825


In [38]:
crosswalk <- read.csv("./data/ca_xwalk.csv")

In [39]:
crosswalk <- subset(crosswalk, select = c("tabblk2010","trct"))

In [40]:
head(crosswalk)

,tabblk2010,trct
,<dbl>,<dbl>
1,60030100002053,6003010000
2,60070013003023,6007001300
3,60150002031279,6015000203
4,60150002031336,6015000203
5,60150002031264,6015000203
6,60150002031342,6015000203


In [15]:
install.packages("sqldf")

Installing package into 'C:/Users/Douglas Schoewe/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

also installing the dependencies 'bit', 'bit64', 'blob', 'plogr', 'gsubfn', 'proto', 'RSQLite', 'chron'




package 'bit' successfully unpacked and MD5 sums checked
package 'bit64' successfully unpacked and MD5 sums checked
package 'blob' successfully unpacked and MD5 sums checked
package 'plogr' successfully unpacked and MD5 sums checked
package 'gsubfn' successfully unpacked and MD5 sums checked
package 'proto' successfully unpacked and MD5 sums checked
package 'RSQLite' successfully unpacked and MD5 sums checked
package 'chron' successfully unpacked and MD5 sums checked
package 'sqldf' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Douglas Schoewe\AppData\Local\Temp\RtmpOcBPM1\downloaded_packages


In [42]:
library(sqldf)

In [43]:
library(rgdal)

In [44]:
tmp <- sqldf('SELECT * from tract_311')

In [45]:
AER_311 <- sqldf('SELECT w_geocode, CNS17 FROM WAC')

In [46]:
head(AER_311)

,w_geocode,CNS17
,<dbl>,<int>
1,60014001001007,0
2,60014001001008,0
3,60014001001017,0
4,60014001001024,0
5,60014001001026,0
6,60014001001027,0


In [47]:
AER_311 <- sqldf('SELECT w_geocode, CNS17 FROM WAC ORDER BY CNS17 DESC')

In [48]:
AER_311 <- sqldf('SELECT w_geocode, CNS17 AS AER FROM WAC')
head(AER_311)

,w_geocode,AER
,<dbl>,<int>
1,60014001001007,0
2,60014001001008,0
3,60014001001017,0
4,60014001001024,0
5,60014001001026,0
6,60014001001027,0


In [49]:
sqldf('SELECT * from tract_311 LIMIT 10')


GEOID10,Category
<dbl>,<chr>
6075980600,Abandoned Vehicle
6075980600,Graffiti Public Property
6075980600,Street and Sidewalk Cleaning
6075980600,Street and Sidewalk Cleaning
6075980600,Street and Sidewalk Cleaning
6075980600,Street and Sidewalk Cleaning
6075980600,Street and Sidewalk Cleaning
6075980600,Street and Sidewalk Cleaning
6075980600,Street and Sidewalk Cleaning


In [65]:
noise_311 <- sqldf('SELECT * from tract_311 WHERE Category = "Noise Report"')

In [69]:
AER_311_Tract <- sqldf("SELECT * from AER_311 AS A LEFT JOIN crosswalk AS B on A.w_geocode = B.tabblk2010") 

In [70]:
head(AER_311_Tract)

,w_geocode,AER,tabblk2010,trct
,<dbl>,<int>,<dbl>,<dbl>
1,60014001001007,0,60014001001007,6001400100
2,60014001001008,0,60014001001008,6001400100
3,60014001001017,0,60014001001017,6001422700
4,60014001001024,0,60014001001024,6001400100
5,60014001001026,0,60014001001026,6001400100
6,60014001001027,0,60014001001027,6001400100


In [71]:
AER_311_Tract <- sqldf("SELECT sum(A.AER) AS AER, B.trct from AER_311 AS A LEFT JOIN crosswalk AS B on A.w_geocode = B.tabblk2010 GROUP BY trct")

In [72]:
head(AER_311_Tract)

,AER,trct
,<int>,<dbl>
1,15,6001400100
2,1,6001400200
3,8,6001400300
4,2,6001400400
5,0,6001400500
6,2,6001400600


In [73]:
noise_311_tract <- sqldf("SELECT GEOID10, count(Category) as Noise_C FROM noise_311 GROUP BY GEOID10")

In [74]:
noise_AER_tract <- merge(AER_311_Tract,noise_311_tract, by.x="trct",by.y = "GEOID10")

In [76]:
head(noise_AER_tract)

,trct,AER,Noise_C
,<dbl>,<int>,<int>
1,6075010100,71,5
2,6075010200,5,6
3,6075010300,27,5
4,6075010400,238,2
5,6075010500,576,10
6,6075010700,56,4


In [77]:
install.packages("RCurl")

Installing package into 'C:/Users/Douglas Schoewe/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

also installing the dependency 'bitops'




package 'bitops' successfully unpacked and MD5 sums checked
package 'RCurl' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Douglas Schoewe\AppData\Local\Temp\RtmpOcBPM1\downloaded_packages


In [78]:
install.packages("RJSONIO")

Installing package into 'C:/Users/Douglas Schoewe/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



package 'RJSONIO' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Douglas Schoewe\AppData\Local\Temp\RtmpOcBPM1\downloaded_packages


In [79]:
install.packages("devtools")

Installing package into 'C:/Users/Douglas Schoewe/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



package 'devtools' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Douglas Schoewe\AppData\Local\Temp\RtmpOcBPM1\downloaded_packages


In [80]:
library(devtools)

Loading required package: usethis



In [81]:
install_github("alexsingleton/cartodb-r", subdir = "CartoDB")


✔  checking for file 'C:\Users\Douglas Schoewe\AppData\Local\Temp\RtmpOcBPM1\remotes6d0835183510\alexsingleton-cartodb-r-ea893c4\CartoDB/DESCRIPTION'
─  preparing 'CartoDB':
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building 'CartoDB_1.4.tar.gz'
   


Installing package into 'C:/Users/Douglas Schoewe/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



In [82]:
library(CartoDB)

Loading required package: RCurl

Loading required package: RJSONIO



In [25]:
##Beyond this point I was unable to connect and read/write to the Carto API## I called Carto and spoke with a representitve## He said that the Free version no longer supports using CARTO API

In [117]:
cartodb("urbananlytics",api.key="5c0dcfdd79366035bf50cc6229da08032b35445a")

In [119]:
install.packages("devtools") 
devtools::install_github("dracodoc/rCartoAPI")

Warning message:
"package 'devtools' is in use and will not be installed"



generics  (NA -> 0.1.3) [CRAN]
lubridate (NA -> 1.8.0) [CRAN]


Installing 2 packages: generics, lubridate

Installing packages into 'C:/Users/Douglas Schoewe/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

Warning message in download.file(url, destfile, method, mode = "wb", ...):
"URL 'https://cran.r-project.org/bin/windows/contrib/4.2/generics_0.1.3.zip': Timeout of 60 seconds was reached"


Error in download.file(url, destfile, method, mode = "wb", ...) : 
  cannot open URL 'https://cran.r-project.org/bin/windows/contrib/4.2/generics_0.1.3.zip'


Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
"download of package 'generics' failed"
Warning message in download.file(url, destfile, method, mode = "wb", ...):
"URL 'https://cran.r-project.org/bin/windows/contrib/4.2/lubridate_1.8.0.zip': Timeout of 60 seconds was reached"


Error in download.file(url, destfile, method, mode = "wb", ...) : 
  cannot open URL 'https://cran.r-project.org/bin/windows/contrib/4.2/lubridate_1.8.0.zip'


Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
"download of package 'lubridate' failed"


✔  checking for file 'C:\Users\Douglas Schoewe\AppData\Local\Temp\RtmpOcBPM1\remotes6d08a787c42\dracodoc-rCartoAPI-7851f2b/DESCRIPTION' (557ms)
─  preparing 'rCartoAPI': (354ms)
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
   Omitted 'LazyData' from DESCRIPTION
─  building 'rCartoAPI_0.1.0.tar.gz'
   


Installing package into 'C:/Users/Douglas Schoewe/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

Warning message in i.p(...):
"installation of package 'C:/Users/DOUGLA~1/AppData/Local/Temp/RtmpOcBPM1/file6d083ea275fa/rCartoAPI_0.1.0.tar.gz' had non-zero exit status"


In [120]:
remotes::install_github("dracodoc/rCartoAPI")

generics  (NA -> 0.1.3) [CRAN]
lubridate (NA -> 1.8.0) [CRAN]


Installing 2 packages: generics, lubridate

Installing packages into 'C:/Users/Douglas Schoewe/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



package 'generics' successfully unpacked and MD5 sums checked
package 'lubridate' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Douglas Schoewe\AppData\Local\Temp\RtmpOcBPM1\downloaded_packages
✔  checking for file 'C:\Users\Douglas Schoewe\AppData\Local\Temp\RtmpOcBPM1\remotes6d082cda3a96\dracodoc-rCartoAPI-7851f2b/DESCRIPTION'
─  preparing 'rCartoAPI':
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
   Omitted 'LazyData' from DESCRIPTION
─  building 'rCartoAPI_0.1.0.tar.gz'
   


Installing package into 'C:/Users/Douglas Schoewe/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



In [13]:
tmp <- cartodb.collection("brooklyn_trees", geomAs="XY", limit=10)

ERROR: Error in cartodb.collection("brooklyn_trees", geomAs = "XY", limit = 10): could not find function "cartodb.collection"


In [14]:
tmp

#Same query, but supplied as a formal SQL query
tmp <- cartodb.collection(sql="SELECT cartodb_id, the_geom, tree_speci, serial FROM brooklyn_trees LIMIT 10")
tmp

ERROR: Error in eval(expr, envir, enclos): object 'tmp' not found


In [15]:
tmp

#Same query, but supplied as a formal SQL query
tmp <- cartodb.collection(sql="SELECT cartodb_id,  ST_AsText(the_geom), ST_X(the_geom) AS Lon, ST_Y(the_geom) AS Lat, tree_speci, serial FROM brooklyn_trees LIMIT 10")
tmp

ERROR: Error in eval(expr, envir, enclos): object 'tmp' not found


In [16]:
#Get the GeoJSON
zones <- cartodb.collection("brooklyn_neigh_tab",columns=c("cartodb_id","the_geom"),method="GeoJSON")
#Create a SpatialPolygonsDataFrame obect
zones.SP <-readOGR(zones,layer = 'OGRGeoJSON',verbose =FALSE) #The "verbose = FALSE" stops readOGR printing the content of the geoJSON to the R console
#Plot the zones
plot(zones.SP)

ERROR: Error in cartodb.collection("brooklyn_neigh_tab", columns = c("cartodb_id", : could not find function "cartodb.collection"


In [17]:
test<- cartodb.collection(sql="SELECT cartodb_id, ntaname, ST_Area(the_geom::geography) / 1000000 AS Area_KM FROM brooklyn_neigh_tab")

ERROR: Error in cartodb.collection(sql = "SELECT cartodb_id, ntaname, ST_Area(the_geom::geography) / 1000000 AS Area_KM FROM brooklyn_neigh_tab"): could not find function "cartodb.collection"


In [18]:
test <- cartodb.collection(sql="SELECT cartodb_id, ntaname, ST_Centroid(the_geom) as the_geom FROM brooklyn_neigh_tab",method="GeoJSON")

# Plot the results
plot(readOGR(test,layer = 'OGRGeoJSON',verbose =FALSE))

ERROR: Error in cartodb.collection(sql = "SELECT cartodb_id, ntaname, ST_Centroid(the_geom) as the_geom FROM brooklyn_neigh_tab", : could not find function "cartodb.collection"


In [19]:
#Return buffers
buffers <- cartodb.collection(sql="SELECT cartodb_id,  ST_Buffer(the_geom::geography,200) as the_geom FROM brooklyn_trees WHERE tree_speci = 'Green Ash'",method="GeoJSON")

# Plot the results
buffers.SP <- readOGR(buffers,layer = 'OGRGeoJSON',verbose =FALSE)
#Plot first 100 tree buffers
plot(buffers.SP[1:100,])

ERROR: Error in cartodb.collection(sql = "SELECT cartodb_id,  ST_Buffer(the_geom::geography,200) as the_geom FROM brooklyn_trees WHERE tree_speci = 'Green Ash'", : could not find function "cartodb.collection"


In [20]:
# Return intersection of stations with Brooklyn neighborhoods
intersection_stations <- cartodb.collection(sql="SELECT a.*,b.ntaname FROM subway_stations AS a, brooklyn_neigh_tab AS b WHERE ST_Intersects(a.the_geom, b.the_geom)",method="GeoJSON") # b.ntaname is added to the select statement to append the neighbouhood names that the stations are located within

# Plot the results
intersection_stations.SP <- readOGR(intersection_stations,layer = 'OGRGeoJSON',verbose =FALSE)
plot(intersection_stations.SP)

#We can also see how each of the stations now have the neighborhood appended
head(intersection_stations.SP@data)

ERROR: Error in cartodb.collection(sql = "SELECT a.*,b.ntaname FROM subway_stations AS a, brooklyn_neigh_tab AS b WHERE ST_Intersects(a.the_geom, b.the_geom)", : could not find function "cartodb.collection"


In [21]:
# Write the query to Carto
cartodb.collection(sql="CREATE TABLE subway_stations_brooklyn AS SELECT a.* FROM subway_stations AS a, brooklyn_neigh_tab AS b WHERE ST_Intersects(a.the_geom, b.the_geom)")

# In order to make the table visible on the datasets page, this statement is also required
cartodb.collection(sql="select cdb_cartodbfytable('subway_stations_brooklyn')")

ERROR: Error in cartodb.collection(sql = "CREATE TABLE subway_stations_brooklyn AS SELECT a.* FROM subway_stations AS a, brooklyn_neigh_tab AS b WHERE ST_Intersects(a.the_geom, b.the_geom)"): could not find function "cartodb.collection"


In [22]:
# Return intersection of stations with Brooklyn neighborhoods
intersection_lines <- cartodb.collection(sql="SELECT a.* FROM subway a, brooklyn_neigh_tab b WHERE ST_Intersects(a.the_geom, b.the_geom)",method="GeoJSON")

# Plot the results
intersection_lines.SP <- readOGR(intersection_lines,layer = 'OGRGeoJSON',verbose =FALSE)
plot(intersection_lines.SP)

ERROR: Error in cartodb.collection(sql = "SELECT a.* FROM subway a, brooklyn_neigh_tab b WHERE ST_Intersects(a.the_geom, b.the_geom)", : could not find function "cartodb.collection"


In [23]:
# Return intersection of stations with Brooklyn neighborhoods
lines <- cartodb.collection(sql="SELECT * FROM subway",method="GeoJSON")

# Plot the results
lines.SP <- readOGR(lines,layer = 'OGRGeoJSON',verbose =FALSE)
plot(lines.SP)
plot(zones.SP, col="#24478f4D", border = NA, add=TRUE)
plot(intersection_lines.SP,col="red", add=TRUE)

ERROR: Error in cartodb.collection(sql = "SELECT * FROM subway", method = "GeoJSON"): could not find function "cartodb.collection"


In [24]:
distance_station <- cartodb.collection(sql="SELECT name, ST_Distance(the_geom::geography, ST_SetSRID(ST_Point(-73.9365832, 40.7042924),4326)::geography) as distance FROM subway_stations_brooklyn ORDER BY distance")

#Show the closest six subway stations
head(distance_station)

ERROR: Error in cartodb.collection(sql = "SELECT name, ST_Distance(the_geom::geography, ST_SetSRID(ST_Point(-73.9365832, 40.7042924),4326)::geography) as distance FROM subway_stations_brooklyn ORDER BY distance"): could not find function "cartodb.collection"
